# Testing operations with PyTorch

In [1]:
import numpy as np
import torch
import random
import parameters as var #Configuration and coarsening parameters
var.init() #initializes parameters
import utils as ut #Some utility functions 
import loss_function as lf #Custom loss function


from opendataset import ConfsDataset #class for opening gauge confs
from opendataset import read_binary_conf
import operators as op #Interpolator and prolongator given a set of test vectors
import operators_torch as opt #Same implementations but with pytorch

In [2]:
def numpy_loss(pred_test_vectors,test_vectors):
    loss = 0
    for i in range(batch_size):
        ops = op.Operators(var.BLOCKS_X,var.BLOCKS_T,pred_test_vectors[i])
        for tv in range(var.NV):
            #Evaluate 
            loss += np.linalg.norm( (test_vectors[i,tv] - ops.P_Pdagg(test_vectors[i,tv])) )
            #loss = np.linealg.norm(test_vectors - P_Pdagg(test_vectors)) #|| . ||₂ (l2-norm)
    return loss

def torch_loss(pred, target):
    """
    pred  : Tensor of shape (B, NV, 2, NT, NX)   (complex numbers stored as complex dtype)
    target: Tensor of shape (B, NV, 2, NT, NX)   (the “near kernel” vectors)
    Returns a scalar loss Tensor (requires_grad=True)
    """
    batch_size = pred.shape[0]
    loss = 0.0
    for i in range(batch_size):
            
        ops = opt.Operators(var.BLOCKS_X, var.BLOCKS_T, pred[i])   
        for tv in range(var.NV):
            corrected = ops.P_Pdagg(target[i, tv])               
            diff = target[i, tv] - corrected
            loss = loss + torch.linalg.norm(diff)   
    return loss

In [3]:
def numpy_loss_ind(pred_test_vectors,test_vectors):
    loss = 0.0
    ops = op.Operators(var.BLOCKS_X,var.BLOCKS_T,pred_test_vectors)
    if test_vectors.shape[0] == var.NV:
        for tv in range(var.NV):
            #Evaluate 
            loss += np.linalg.norm( (test_vectors[tv] - ops.P_Pdagg(test_vectors[tv])) )/np.linalg.norm(test_vectors[tv])
            #loss = np.linealg.norm(test_vectors - P_Pdagg(test_vectors)) #|| . ||₂ (l2-norm)
    else:
        loss += np.linalg.norm( (test_vectors - ops.P_Pdagg(test_vectors)) )/np.linalg.norm(test_vectors)
    return loss

def torch_loss_ind(pred, target):
    loss = 0.0      
    ops = opt.Operators(var.BLOCKS_X, var.BLOCKS_T, pred)
    if target.shape[0] == var.NV:
        for tv in range(var.NV):
            corrected = ops.P_Pdagg(target[tv])               
            diff = target[tv] - corrected
            loss = loss + torch.linalg.norm(diff)/torch.linalg.norm(target[tv])
    else:
        loss += torch.linalg.norm( (target - ops.P_Pdagg(target)) ) / torch.linalg.norm(target)
    return loss

In [4]:
"""
Loading the configurations and the near-kernel test vectors
"""
workers = 4
batch_size = 300
dataset = ConfsDataset()
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                shuffle=False, num_workers=workers)

#----returns a tensor of size [ [batch_size,4,Nx,Nt], [batch_size,Nv,2,Nx,Nt] ]----#
first_batch = next(iter(dataloader)) 
#--------------------------------------

In [5]:
#For testing torch vs numpy
test = first_batch[1][0]

print("set of test vectors",type(test))
oper = op.Operators(var.BLOCKS_X, var.BLOCKS_T,test)
tvecs = oper.getTestVectors()
print("numpy vectors",type(tvecs))
oper_torch = opt.Operators(var.BLOCKS_X, var.BLOCKS_T,test)
tvecs_torch = oper_torch.getTestVectors()
print("torch vectors",type(tvecs_torch))
print(tvecs.shape,tvecs_torch.shape)
print("test vectors",tvecs[0,1,2,2],tvecs_torch[0,1,2,2])

v = np.random.rand(2,var.NT,var.NX) + 1j*np.random.rand(2,var.NT,var.NX)
vc = np.random.rand(var.NV,2,var.BLOCKS_T,var.BLOCKS_X) + 1j*np.random.rand(var.NV,2,var.BLOCKS_T,var.BLOCKS_X)

vtorch = torch.tensor(v)
vctorch = torch.tensor(vc)


out = oper.P_vc(vc)
print("Pvc (np)",out[0,5,0],type(out))
outt = oper_torch.P_vc(vctorch)
print("Pvc (torch)",outt[0,5,0],type(outt))

out = oper.P_Pdagg(v)
print("PP^+ (np)",v[0,5,0],out[0,5,0])
outvc = oper.Pdagg_P(vc)
print("P^+P (np)",vc[0,0,0,0],outvc[0,0,0,0])

outt = oper_torch.P_Pdagg(vtorch)
print("PP^+ (torch)",vtorch[0,5,0],outt[0,5,0])
outvct = oper_torch.Pdagg_P(vctorch)
print("P^+P (torch)",vctorch[0,0,0,0],outvct[0,0,0,0])

set of test vectors <class 'torch.Tensor'>
numpy vectors <class 'numpy.ndarray'>
torch vectors <class 'torch.Tensor'>
(20, 2, 32, 32) torch.Size([20, 2, 32, 32])
test vectors (0.028727087023567598-0.0003883502939103646j) tensor(0.0287-0.0004j, dtype=torch.complex128)
Pvc (np) (0.0939966431262407-0.04463742653733617j) <class 'numpy.ndarray'>
Pvc (torch) tensor(0.0940-0.0446j, dtype=torch.complex128) <class 'torch.Tensor'>
PP^+ (np) (0.6488143409106863+0.9406873482204082j) (0.06210505937746227-0.09254589311067246j)
P^+P (np) (0.0055086926849828854+0.026555372987494774j) (0.0055086926849828854+0.02655537298749494j)
PP^+ (torch) tensor(0.6488+0.9407j, dtype=torch.complex128) tensor(0.0621-0.0925j, dtype=torch.complex128)
P^+P (torch) tensor(0.0055+0.0266j, dtype=torch.complex128) tensor(0.0055+0.0266j, dtype=torch.complex128)


In [6]:
random.seed(4)
test_vectors = np.zeros((var.NV,2,var.NT,var.NX),dtype=complex)
for tv in range(var.NV):
    for nt in range(var.NT):
        for nx in range(var.NX):
            for s in range(2):
                x = random.random()
                y = random.random()
                z = complex(x, y)
                test_vectors[tv,s,nt,nx] = z   #2*(nx*NT + nt) + s + 1 + tv*2*NX*NT

In [7]:
print(numpy_loss_ind(first_batch[1][0].numpy(),first_batch[1][0].numpy()))
print(torch_loss_ind(first_batch[1][0],first_batch[1][0]))

9.446731534055534e-15
tensor(1.1535e-14, dtype=torch.float64)


In [8]:
#This checks whether random test vectors are contained on the image of P when this operator is 
#assembled with the test vectors from SAP
print(numpy_loss_ind(first_batch[1][0].numpy(),test_vectors)/var.NV)
print(torch_loss_ind(first_batch[1][0],torch.tensor(test_vectors))/var.NV)

0.9605938177860069
tensor(0.9606, dtype=torch.float64)


In [9]:
#This does the opposite: checks whether the test vectors from SAP are contained in the image of P when built
#with random vectors
print(numpy_loss_ind(test_vectors,first_batch[1][0].numpy())/var.NV)
print(torch_loss_ind(torch.tensor(test_vectors),first_batch[1][2])/var.NV)

0.9607551160050389
tensor(0.9628, dtype=torch.float64)


In [10]:
#Checking that the test vectors generated with SAP are approximately contained in the range of P
loss = 0.0
remTV = 30-var.NV
for confID in range(batch_size):
    print("confID",confID)
    for i in range(remTV):
        tvector = np.zeros((2,var.NT,var.NX),dtype=complex)
        path = 'sap/near_kernel/b{0}_{1}x{2}/{3}/tvector_{1}x{2}_b{0}0000_m{4}_nconf{5}_tv{6}.tv'.format(
        int(var.BETA),var.NX,var.NT,var.M0_FOLDER,var.M0_STRING,confID,29-i)
        tvector = read_binary_conf(None,path)
        print("Test vector ",29-i)
        loss += numpy_loss_ind(first_batch[1][confID].numpy(),tvector)
        print("Numpy",numpy_loss_ind(first_batch[1][confID].numpy(),tvector))
        print("PyTorch",torch_loss_ind(first_batch[1][confID],torch.tensor(tvector)))
    print("----------------------")
print("Final loss on the remainder test vectors",loss/(batch_size*remTV))

confID 0
Test vector  29
Numpy 0.11432883126010535
PyTorch tensor(0.1143, dtype=torch.float64)
Test vector  28
Numpy 0.10667270388878768
PyTorch tensor(0.1067, dtype=torch.float64)
Test vector  27
Numpy 0.08226299875343232
PyTorch tensor(0.0823, dtype=torch.float64)
Test vector  26
Numpy 0.11903211612904607
PyTorch tensor(0.1190, dtype=torch.float64)
Test vector  25
Numpy 0.10968158294603861
PyTorch tensor(0.1097, dtype=torch.float64)
Test vector  24
Numpy 0.08347698142094806
PyTorch tensor(0.0835, dtype=torch.float64)
Test vector  23
Numpy 0.10708122902465027
PyTorch tensor(0.1071, dtype=torch.float64)
Test vector  22
Numpy 0.09006190909662214
PyTorch tensor(0.0901, dtype=torch.float64)
Test vector  21
Numpy 0.10257302249412367
PyTorch tensor(0.1026, dtype=torch.float64)
Test vector  20
Numpy 0.09187777709564274
PyTorch tensor(0.0919, dtype=torch.float64)
----------------------
confID 1
Test vector  29
Numpy 0.0745687296590846
PyTorch tensor(0.0746, dtype=torch.float64)
Test vector  2

Final loss on the remainder test vectors 9.386980374900652 (for 4 test vectors and using the rest to check the loss function)
Final loss on the remainder test vectors 2.5979268393039137 (for 14 test vectors and using the rest to check the loss function)

This shows that the more smoothed test vectors, the better they capture other near-kernel components In other terms, the more smoothed test vectors I use, the smaller the metric.

Just out of curiosity. What happens if I evaluate the metric just on random vectors?

In [11]:
#Using the test vectors from SAP to build P, P^+ and evaluate on random test vectors.
loss = 0.0
for confID in range(batch_size):
    print("confID",confID)
    for i in range(var.NV):
        tvector = np.random.rand(2,var.NT,var.NX) + 1j*np.random.rand(2,var.NT,var.NX)
        print("rand vector",i)
        loss += numpy_loss_ind(first_batch[1][confID].numpy(),tvector)
        print("Numpy",numpy_loss_ind(first_batch[1][confID].numpy(),tvector))
        print("PyTorch",torch_loss_ind(first_batch[1][confID],torch.tensor(tvector)))
    print("----------------------")
print("Final loss on the remainder test vectors",loss/(batch_size*var.NV))

confID 0
rand vector 0
Numpy 0.9650607407940232
PyTorch tensor(0.9651, dtype=torch.float64)
rand vector 1
Numpy 0.9630909171665016
PyTorch tensor(0.9631, dtype=torch.float64)
rand vector 2
Numpy 0.9584832262433601
PyTorch tensor(0.9585, dtype=torch.float64)
rand vector 3
Numpy 0.9602176016631165
PyTorch tensor(0.9602, dtype=torch.float64)
rand vector 4
Numpy 0.9614786944736976
PyTorch tensor(0.9615, dtype=torch.float64)
rand vector 5
Numpy 0.9602254572150262
PyTorch tensor(0.9602, dtype=torch.float64)
rand vector 6
Numpy 0.9618855916266573
PyTorch tensor(0.9619, dtype=torch.float64)
rand vector 7
Numpy 0.9615382208848644
PyTorch tensor(0.9615, dtype=torch.float64)
rand vector 8
Numpy 0.964127013367503
PyTorch tensor(0.9641, dtype=torch.float64)
rand vector 9
Numpy 0.9584955104294096
PyTorch tensor(0.9585, dtype=torch.float64)
rand vector 10
Numpy 0.9601777519819691
PyTorch tensor(0.9602, dtype=torch.float64)
rand vector 11
Numpy 0.9589104687921812
PyTorch tensor(0.9589, dtype=torch.flo

In [12]:
#Checking how well random vectors contain the SAP test vectors
test_vectors = np.zeros((var.NV,2,var.NT,var.NX),dtype=complex)
#random.seed(0)
for tv in range(var.NV):
    for nt in range(var.NT):
        for nx in range(var.NX):
            for s in range(2):
                x = random.random()
                y = random.random()
                z = complex(x, y)
                test_vectors[tv,s,nt,nx] = z   #2*(nx*NT + nt) + s + 1 + tv*2*NX*NT

loss = 0.0
for confID in range(batch_size):
    print("confID",confID)
    loss += numpy_loss_ind(test_vectors,first_batch[1][confID].numpy())
    print("Numpy",numpy_loss_ind(test_vectors,first_batch[1][confID].numpy()))
    print("PyTorch",torch_loss_ind(torch.tensor(test_vectors),first_batch[1][confID]))
    print("----------------------")
print("Final loss on the remainder test vectors",loss/(batch_size*var.NV))

confID 0
Numpy 19.224820334070355
PyTorch tensor(19.2248, dtype=torch.float64)
----------------------
confID 1
Numpy 19.166590314438047
PyTorch tensor(19.1666, dtype=torch.float64)
----------------------
confID 2
Numpy 19.204563874642712
PyTorch tensor(19.2046, dtype=torch.float64)
----------------------
confID 3
Numpy 19.240943655109234
PyTorch tensor(19.2409, dtype=torch.float64)
----------------------
confID 4
Numpy 19.217685831835755
PyTorch tensor(19.2177, dtype=torch.float64)
----------------------
confID 5
Numpy 19.225565704233002
PyTorch tensor(19.2256, dtype=torch.float64)
----------------------
confID 6
Numpy 19.220201427456733
PyTorch tensor(19.2202, dtype=torch.float64)
----------------------
confID 7
Numpy 19.219628004750977
PyTorch tensor(19.2196, dtype=torch.float64)
----------------------
confID 8
Numpy 19.165609814940076
PyTorch tensor(19.1656, dtype=torch.float64)
----------------------
confID 9
Numpy 19.14139195661035
PyTorch tensor(19.1414, dtype=torch.float64)
----